# LAB Monday: Scrape the current top 100 songs and their respective artists, and put the information into a pandas dataframe.

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url='https://www.billboard.com/charts/hot-100/'

In [3]:
results=requests.get(url, headers ={'Accept-Language':'en-US'})

In [4]:
soup = BeautifulSoup(results.content,'html.parser')

In [5]:
soup.select('h3.c-title.a-no-trucate')[0].get_text(strip=True)

"We Don't Talk About Bruno"

In [6]:
soup.select('span.c-label.a-no-trucate')[0].get_text(strip=True)

'Carolina Gaitan, Mauro Castillo, Adassa, Rhenzy Feliz, Diane Guerrero, Stephanie Beatriz & Encanto Cast'

In [7]:
title=[]
artists=[]

len_songs=len(soup.select('h3.c-title.a-no-trucate'))

In [8]:
len_songs

100

In [9]:
# title_df=[]
# artist_df=[]

# for songs in title:
#     title_df.append(songs.replace('\n', ''))

# for songs in artists:
#     artist_df.append(songs.replace('\n', ''))


# using strip=True in the step before it is not necessary to replace \n

In [10]:
len(title)

0

In [11]:
len(artists)

0

In [12]:
import pandas as pd

songs=pd.DataFrame({'Song_title':title, 'Artists':artists})
songs

,Song_title,Artists


In [13]:
import numpy as np
songs['Rank']=np.arange(len(songs))+1

In [14]:
songs

,Song_title,Artists,Rank


# LAB Tuesday: Prototype with user input

- collect user input with a prompt eg. 'what song do you like?'


- deal with at least one possible user input error / behaviour (you can add more issues and resolutions later) eg. standardise upper/lower case
- look for what the user has input, against your 100 hot song collection (more flexible lookup can be added later)
- feedback to user if their song is hot or not. 

- provide a recommendation from the collection - potentially randomly

# LAB Wednesday: API wrappers - Create your collection of songs & audio features

- Instructions

To move forward with the project, you need to create a collection of songs with their audio features - as large as possible!

These are the songs that we will cluster. And, later, when the user inputs a song, we will find the cluster to which the song belongs and recommend a song from the same cluster. The more songs you have, the more accurate and diverse recommendations you'll be able to give. Although... you might want to make sure the collected songs are "curated" in a certain way. Try to find playlists of songs that are diverse, but also that meet certain standards.

The process of sending hundreds or thousands of requests can take some time - it's normal if you have to wait a few minutes (or, if you're ambitious, even hours) to get all the data you need.

An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!

In [15]:
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials
import getpass

In [16]:
client_id = str(getpass.getpass('client_id?'))
client_secret = str(getpass.getpass('client_secret?'))

client_id?········
client_secret?········


In [17]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [18]:
playlist=sp.user_playlist_tracks('Bradnumber1','1YL4XoegERoragv0RK2RC9')

In [19]:
playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [20]:
playlist['items'][4]['track']['id']

'4QlBUvcyODBn0ukZoYb7v1'

In [21]:
playlist['total']

9974

In [22]:
len(playlist['items'])

100

In [23]:
def get_playlist_tracks(user_id,playlist_id):
    results = sp.user_playlist_tracks(user_id,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
    tracks.extend(results['items'])
    return tracks

In [24]:
full_track_ids=get_playlist_tracks('Bradnumber1','1YL4XoegERoragv0RK2RC9')

In [25]:
len(full_track_ids)

174

In [26]:
full_track_ids[0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [27]:
full_track_ids[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [28]:
full_track_ids[0]['track']['name']

'Seventh Son'

In [29]:
full_track_ids[0]['track']['artists'][0]['name']

'Johnny Rivers'

In [30]:
full_track_ids[0]['track']['id']

'6LBueuTAVmHEEezP2cnnZB'

In [31]:
title=[]
artist=[]
song_id=[]

for i in full_track_ids:
    title.append(i['track']['name'])
    artist.append(i['track']['artists'][0]['name'])
    song_id.append(i['track']['id'])

In [32]:
playlist_df=pd.DataFrame({'title': title, 'artist': artist, 'song_id': song_id})

In [33]:
playlist_df

,title,artist,song_id
0,Seventh Son,Johnny Rivers,6LBueuTAVmHEEezP2cnnZB
1,Nobody's Child,Karen Young,0cA0t6FxX2FZkX0BKZnUTa
2,Mustang Sally,The Commitments,1XlCCgKKrQQfXaDyv4GrZe
3,Personality,Lloyd Price,5vjigjfoXuJEttqLB3HTAg
4,Good Morning Starshine,Oliver,4QlBUvcyODBn0ukZoYb7v1
...,...,...,...
169,Sick Cycle Carousel,Lifehouse,1IMi3chVPe4t5MUxIjn4Lk
170,Kings & Queens,Ava Max,1InCpEliLCkWHOBdXW7XSx
171,Believer,Guy Sebastian,5CvWCItJ13iDRH2bEZhtjy
172,Aussie As,Matt Scullion,47DzE0bNSqWj4z3mrPU3gj


In [34]:
spaceplaylist=[]
for i in range(len(playlist_df)):
    spaceplaylist.extend(sp.audio_features(tracks=playlist_df['song_id'][i]))

In [35]:
song_features=pd.DataFrame.from_dict(spaceplaylist)

In [36]:
song_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.462,0.714,9,-9.702,1,0.0706,0.58200,0.000002,0.7630,0.890,147.778,audio_features,6LBueuTAVmHEEezP2cnnZB,spotify:track:6LBueuTAVmHEEezP2cnnZB,https://api.spotify.com/v1/tracks/6LBueuTAVmHE...,https://api.spotify.com/v1/audio-analysis/6LBu...,173035,4
1,0.520,0.417,9,-10.809,1,0.0282,0.76200,0.000000,0.1790,0.667,106.230,audio_features,0cA0t6FxX2FZkX0BKZnUTa,spotify:track:0cA0t6FxX2FZkX0BKZnUTa,https://api.spotify.com/v1/tracks/0cA0t6FxX2FZ...,https://api.spotify.com/v1/audio-analysis/0cA0...,263347,4
2,0.744,0.735,1,-10.766,1,0.0587,0.01100,0.000000,0.0564,0.891,115.062,audio_features,1XlCCgKKrQQfXaDyv4GrZe,spotify:track:1XlCCgKKrQQfXaDyv4GrZe,https://api.spotify.com/v1/tracks/1XlCCgKKrQQf...,https://api.spotify.com/v1/audio-analysis/1XlC...,241867,4
3,0.591,0.454,5,-8.936,1,0.0322,0.74400,0.000000,0.0989,0.865,128.909,audio_features,5vjigjfoXuJEttqLB3HTAg,spotify:track:5vjigjfoXuJEttqLB3HTAg,https://api.spotify.com/v1/tracks/5vjigjfoXuJE...,https://api.spotify.com/v1/audio-analysis/5vji...,157733,4
4,0.648,0.560,3,-9.175,0,0.1470,0.49100,0.000000,0.3570,0.698,126.517,audio_features,4QlBUvcyODBn0ukZoYb7v1,spotify:track:4QlBUvcyODBn0ukZoYb7v1,https://api.spotify.com/v1/tracks/4QlBUvcyODBn...,https://api.spotify.com/v1/audio-analysis/4QlB...,218467,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,0.445,0.800,8,-5.228,0,0.0301,0.00168,0.000001,0.0802,0.470,95.481,audio_features,1IMi3chVPe4t5MUxIjn4Lk,spotify:track:1IMi3chVPe4t5MUxIjn4Lk,https://api.spotify.com/v1/tracks/1IMi3chVPe4t...,https://api.spotify.com/v1/audio-analysis/1IMi...,263400,4
170,0.637,0.690,1,-4.057,0,0.0405,0.00786,0.000000,0.1240,0.457,129.857,audio_features,1InCpEliLCkWHOBdXW7XSx,spotify:track:1InCpEliLCkWHOBdXW7XSx,https://api.spotify.com/v1/tracks/1InCpEliLCkW...,https://api.spotify.com/v1/audio-analysis/1InC...,162399,4
171,0.376,0.505,9,-5.534,1,0.0353,0.29600,0.000000,0.0846,0.299,177.794,audio_features,5CvWCItJ13iDRH2bEZhtjy,spotify:track:5CvWCItJ13iDRH2bEZhtjy,https://api.spotify.com/v1/tracks/5CvWCItJ13iD...,https://api.spotify.com/v1/audio-analysis/5CvW...,225493,3
172,0.530,0.340,2,-9.308,1,0.0279,0.57400,0.000002,0.1440,0.231,76.959,audio_features,47DzE0bNSqWj4z3mrPU3gj,spotify:track:47DzE0bNSqWj4z3mrPU3gj,https://api.spotify.com/v1/tracks/47DzE0bNSqWj...,https://api.spotify.com/v1/audio-analysis/47Dz...,219840,5


In [37]:
playlist_features = playlist_df.merge(song_features, left_on='song_id', right_on='id')

In [38]:
playlist_features

,title,artist,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Seventh Son,Johnny Rivers,6LBueuTAVmHEEezP2cnnZB,0.462,0.714,9,-9.702,1,0.0706,0.58200,...,0.7630,0.890,147.778,audio_features,6LBueuTAVmHEEezP2cnnZB,spotify:track:6LBueuTAVmHEEezP2cnnZB,https://api.spotify.com/v1/tracks/6LBueuTAVmHE...,https://api.spotify.com/v1/audio-analysis/6LBu...,173035,4
1,Nobody's Child,Karen Young,0cA0t6FxX2FZkX0BKZnUTa,0.520,0.417,9,-10.809,1,0.0282,0.76200,...,0.1790,0.667,106.230,audio_features,0cA0t6FxX2FZkX0BKZnUTa,spotify:track:0cA0t6FxX2FZkX0BKZnUTa,https://api.spotify.com/v1/tracks/0cA0t6FxX2FZ...,https://api.spotify.com/v1/audio-analysis/0cA0...,263347,4
2,Mustang Sally,The Commitments,1XlCCgKKrQQfXaDyv4GrZe,0.744,0.735,1,-10.766,1,0.0587,0.01100,...,0.0564,0.891,115.062,audio_features,1XlCCgKKrQQfXaDyv4GrZe,spotify:track:1XlCCgKKrQQfXaDyv4GrZe,https://api.spotify.com/v1/tracks/1XlCCgKKrQQf...,https://api.spotify.com/v1/audio-analysis/1XlC...,241867,4
3,Personality,Lloyd Price,5vjigjfoXuJEttqLB3HTAg,0.591,0.454,5,-8.936,1,0.0322,0.74400,...,0.0989,0.865,128.909,audio_features,5vjigjfoXuJEttqLB3HTAg,spotify:track:5vjigjfoXuJEttqLB3HTAg,https://api.spotify.com/v1/tracks/5vjigjfoXuJE...,https://api.spotify.com/v1/audio-analysis/5vji...,157733,4
4,Good Morning Starshine,Oliver,4QlBUvcyODBn0ukZoYb7v1,0.648,0.560,3,-9.175,0,0.1470,0.49100,...,0.3570,0.698,126.517,audio_features,4QlBUvcyODBn0ukZoYb7v1,spotify:track:4QlBUvcyODBn0ukZoYb7v1,https://api.spotify.com/v1/tracks/4QlBUvcyODBn...,https://api.spotify.com/v1/audio-analysis/4QlB...,218467,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Sick Cycle Carousel,Lifehouse,1IMi3chVPe4t5MUxIjn4Lk,0.445,0.800,8,-5.228,0,0.0301,0.00168,...,0.0802,0.470,95.481,audio_features,1IMi3chVPe4t5MUxIjn4Lk,spotify:track:1IMi3chVPe4t5MUxIjn4Lk,https://api.spotify.com/v1/tracks/1IMi3chVPe4t...,https://api.spotify.com/v1/audio-analysis/1IMi...,263400,4
170,Kings & Queens,Ava Max,1InCpEliLCkWHOBdXW7XSx,0.637,0.690,1,-4.057,0,0.0405,0.00786,...,0.1240,0.457,129.857,audio_features,1InCpEliLCkWHOBdXW7XSx,spotify:track:1InCpEliLCkWHOBdXW7XSx,https://api.spotify.com/v1/tracks/1InCpEliLCkW...,https://api.spotify.com/v1/audio-analysis/1InC...,162399,4
171,Believer,Guy Sebastian,5CvWCItJ13iDRH2bEZhtjy,0.376,0.505,9,-5.534,1,0.0353,0.29600,...,0.0846,0.299,177.794,audio_features,5CvWCItJ13iDRH2bEZhtjy,spotify:track:5CvWCItJ13iDRH2bEZhtjy,https://api.spotify.com/v1/tracks/5CvWCItJ13iD...,https://api.spotify.com/v1/audio-analysis/5CvW...,225493,3
172,Aussie As,Matt Scullion,47DzE0bNSqWj4z3mrPU3gj,0.530,0.340,2,-9.308,1,0.0279,0.57400,...,0.1440,0.231,76.959,audio_features,47DzE0bNSqWj4z3mrPU3gj,spotify:track:47DzE0bNSqWj4z3mrPU3gj,https://api.spotify.com/v1/tracks/47DzE0bNSqWj...,https://api.spotify.com/v1/audio-analysis/47Dz...,219840,5
